In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math



from numpy import array
from numpy import argmax
from numpy import mean
from numpy import std


import os
import os.path

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
resposible_residue = 'K'
directory = "Methylation.elm"

In [4]:
full_data = pd.read_table(directory)

In [5]:
full_data.head(10)

,PLMD ID,Uniprot Accession,Position,Type,Sequence,Species,PMIDs
0,PLMD-4,O00139,161,Methylation,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,Homo sapiens,23644510;25514926
1,PLMD-4,O00139,169,Methylation,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,Homo sapiens,23644510;25514926
2,PLMD-9,O00159,383,Methylation,MALQVELVPTGEIIRVVHPHRPCKLALGSDGVRVTMESALTARDRV...,Homo sapiens,23161681
3,PLMD-20,O00193,179,Methylation,MSAARESHPHGVKRSASPDDDLGSSNWEAADLGNEERKQKFLRLMG...,Homo sapiens,25514926
4,PLMD-25,O00217,49,Methylation,MRCLTTPMLLRALAQAARAGPPGGRSLHSSAVAATYKYVNMQDPEM...,Homo sapiens,23748837
5,PLMD-40,O00257,249,Methylation,MELPAVGEHVFAVESIEKKRIRKGRVEYLVKWRGWSPKYNTWEPEE...,Homo sapiens,25514926
6,PLMD-64,O00411,28,Methylation,MSALCWGRGAAGLKRALRPCGRPGLPGKEGTAGGVCGPRRSSSASP...,Homo sapiens,23644510;25514926
7,PLMD-72,O00443,280,Methylation,MAQISSNSGFKECPSSHPEPTRAKDVDKEEALQMEAEALAKLQKDR...,Homo sapiens,25514926
8,PLMD-86,O00482,173,Methylation,MSSNSDTGDLQESLKHGLTPIGAGLPDRHGSPIPARGRLVMLPKVE...,Homo sapiens,25514926
9,PLMD-87,O00483,10,Methylation,MLRQIIGQAKKHPSLIPLFVFIGTGATGATLYLLRLALFNPDVCWD...,Homo sapiens,25514926


In [6]:
full_data.shape

(6323, 7)

In [8]:
WINDOW_SIZE=25

def get_pssm(protein, protein_sz, ind):
  ret = ''
  cnt = 0
  for val in range(ind - WINDOW_SIZE, ind + WINDOW_SIZE + 1):
      now = ''
      if val < 0 or val >= protein_sz:
        # distance = ind - val
        # now = ind + distance
        now = 'X'
        cnt += 1
      else:
        now = protein[val]

      ret += now

  new_protein = protein[0: max(ind-WINDOW_SIZE, 0)] + ret + protein[min(protein_sz, ind+WINDOW_SIZE+1):]
  return cnt, new_protein

In [9]:
res = []
for ind, v in full_data.iterrows():
  plmd = v['PLMD ID']
  pos = v['Position'] - 1
  seq = v['Sequence']
  padX, padded = get_pssm(seq, len(seq), pos)
  now = [plmd, pos, seq, padded, padX]
  res.append(now)

In [10]:
padded_df = pd.DataFrame(res, columns=['PLMD ID', 'Position', 'Sequence', 'Padded Sequence', 'PadX Counter'])

In [11]:
padded_df.head()

,PLMD ID,Position,Sequence,Padded Sequence,PadX Counter
0,PLMD-4,160,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,0
1,PLMD-4,168,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIEN...,0
2,PLMD-9,382,MALQVELVPTGEIIRVVHPHRPCKLALGSDGVRVTMESALTARDRV...,MALQVELVPTGEIIRVVHPHRPCKLALGSDGVRVTMESALTARDRV...,0
3,PLMD-20,178,MSAARESHPHGVKRSASPDDDLGSSNWEAADLGNEERKQKFLRLMG...,MSAARESHPHGVKRSASPDDDLGSSNWEAADLGNEERKQKFLRLMG...,21
4,PLMD-25,48,MRCLTTPMLLRALAQAARAGPPGGRSLHSSAVAATYKYVNMQDPEM...,MRCLTTPMLLRALAQAARAGPPGGRSLHSSAVAATYKYVNMQDPEM...,0


In [12]:
padded_df.tail()

,PLMD ID,Position,Sequence,Padded Sequence,PadX Counter
6318,PLMD-51997,96,XHPVAVASKKRKKPSRPAKTRKRQVGPQSGEVRKEAPRDETKADTD...,XHPVAVASKKRKKPSRPAKTRKRQVGPQSGEVRKEAPRDETKADTD...,0
6319,PLMD-52012,1931,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,0
6320,PLMD-52012,1933,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,0
6321,PLMD-52012,1944,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,MPQRATLLDLSESSAPNLADADDRKWIPPQRSRPILPGPQVLAAMS...,0
6322,PLMD-52160,185,MFRLNSLSALAELAVGSRWYHGGSQPIQIRRRLMMVAFLGASAVTA...,MFRLNSLSALAELAVGSRWYHGGSQPIQIRRRLMMVAFLGASAVTA...,0


In [13]:
f= open("protein.fasta","w+")

for index, row in padded_df.iterrows():
  protein = row['PLMD ID']
  seq = row['Padded Sequence']
  f.write(">" + protein + "\n"+ seq +"\n")

f.close()

In [14]:
f= open("protein_counter.fasta","w+")

for index, row in padded_df.iterrows():
  protein = row['PLMD ID']
  seq = row['Padded Sequence']
  count = row['PadX Counter']
  f.write(">" + protein + " " + str(count) + "\n"+ seq +"\n")

f.close()